In [1]:
# crawl link - buying
from bs4 import BeautifulSoup
import requests

# Khởi tạo một danh sách để lưu trữ các URL
all_urls = []

# Đặt User-Agent để giả mạo trình duyệt
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Sử dụng vòng lặp để duyệt qua các trang
for i in range(1, 25):
    url = f'https://meeyland.com/mua-ban-nha-dat?page={i}'
    
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url, headers=headers)
    
    # Kiểm tra trạng thái phản hồi
    if response.status_code != 200:
        print(f"Error: Unable to access page {i}")
        continue
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm tất cả các thẻ div có class là "relative slider-hover"
    ads = soup.find_all('div', class_='relative slider-hover')
    
    if not ads:
        print(f"No ads found on page {i}")
        continue
    
    # Lấy các liên kết từ thẻ <a>
    for ad in ads:
        link = ad.find('a', href=True)
        if link:
            href = link['href']
            full_url = f'https://meeyland.com{href}'
            all_urls.append(full_url)

# Ghi các URL vào một file văn bản
file_path = "buy_urls.txt"
with open(file_path, "w") as file:
    for url in all_urls:
        file.write(url + "\n")

print("Đã lưu các URL vào file buy_urls.txt")


Đã lưu các URL vào file buy_urls.txt


In [2]:
# crawl link -renting
from bs4 import BeautifulSoup
import requests

# Khởi tạo một danh sách để lưu trữ các URL
all_urls = []

# Đặt User-Agent để giả mạo trình duyệt
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Sử dụng vòng lặp để duyệt qua các trang
for i in range(1, 15):
    url = f'https://meeyland.com/cho-thue-nha-dat?page={i}'
    
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url, headers=headers)
    
    # Kiểm tra trạng thái phản hồi
    if response.status_code != 200:
        print(f"Error: Unable to access page {i}")
        continue
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm tất cả các thẻ div có class là "relative slider-hover"
    ads = soup.find_all('div', class_='relative slider-hover')
    
    if not ads:
        print(f"No ads found on page {i}")
        continue
    
    # Lấy các liên kết từ thẻ <a>
    for ad in ads:
        link = ad.find('a', href=True)
        if link:
            href = link['href']
            full_url = f'https://meeyland.com{href}'
            all_urls.append(full_url)

# Ghi các URL vào một file văn bản
file_path = "rent_urls.txt"
with open(file_path, "w") as file:
    for url in all_urls:
        file.write(url + "\n")

print("Đã lưu các URL vào file rent_urls.txt")



Đã lưu các URL vào file rent_urls.txt


In [ ]:
# crawl data - buying
import requests
from bs4 import BeautifulSoup
import json
import re

# Hàm để lấy dữ liệu từ URL và trích xuất thông tin từ HTML
def extract_data_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }  # Set a User-Agent header to mimic a real browser

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
    
            # Helper function to extract text or return None
            def get_text_or_none(element):
                return element.text.strip() if element else None

            title_element = soup.select_one('h1.text-xl.font-medium')
            title = get_text_or_none(title_element)
            
            address_element = soup.find('p', class_='line-clamp-2 underline lg:no-underline')
            address = get_text_or_none(address_element)
            
            price_element = soup.find('h5', class_='text-black-v8 text-xl font-semibold')
            price = get_text_or_none(price_element)            
            
            description_element = soup.find('div', class_='text-base text-black-v8 leading-[1.875rem]')
            allowed_tags = ['br', 'p']
            for tag in description_element.find_all(True):  # Tìm tất cả các thẻ con
                if tag.name not in allowed_tags:  # Nếu tên thẻ không nằm trong danh sách được phép
                    tag.unwrap()  # Loại bỏ thẻ nhưng giữ lại nội dung
            description = description_element.decode_contents()
            
            phone_pattern = r'(\d{3}[\s\.]?\d{3}[\s\.]?\d{3})'  # Mã số điện thoại có 10 hoặc 11 chữ số
            matches = re.findall(phone_pattern, description)

            if matches:
                phone_number = matches[-1]  # Lấy số điện thoại cuối cùng trong danh sách
            else:
                phone_number = '0349208325'
                
            basic_info = {
                info.find('p').text.strip().replace(':', ''): get_text_or_none(info.find('p', class_='font-medium line-clamp-2'))
                for info in soup.find_all('div', class_='flex text-base text-black-v8 py-2 space-x-8')
            }
            
            # Chỉnh sửa phần diện tích
            area = basic_info.get('Diện tích', None)
            area = area.split('(')[0].strip() if area else None
            
            # Lấy link ảnh đầu tiên
            image_element = soup.select_one('div.image-fill-wrapper img')
            image_link = image_element['src'] if image_element else None
            
            data = {
                'title': title,
                'address': address,
                'price': price,
                'description': description,
                'type_estate': basic_info.get('Loại nhà đất', None),
                'type': 'Mua Bán',
                'area': area,
                'bedroom': basic_info.get('Số phòng ngủ', None),
                'toilet': basic_info.get('Số phòng tắm/ Toilet', None),
                'law': basic_info.get('Giấy tờ pháp lý', None),
                'direction': basic_info.get('Hướng nhà', None),
                'width': basic_info.get('Mặt tiền', None),
                'floor': basic_info.get('Số tầng', None),
                'image_links': image_link,
                "phonenumber": phone_number
            }
            
            return data
        else:
            print(f"Lỗi khi truy cập vào URL: {url}, Status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Lỗi mạng hoặc yêu cầu: {e}")
        return None
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ URL: {url}, Lỗi: {e}")
        return None

# Read URLs from meeyland_urls.txt file
with open('buy_urls.txt', 'r') as file:
    urls = file.read().splitlines()

# Loop through each URL, extract data, and save to the list
data_list = []
for url in urls:
    data = extract_data_from_url(url)
    if data:
        data_list.append(data)
    # time.sleep(2)  # Add a delay between requests to avoid overwhelming the server

# Write data to JSON file
with open('buy_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu vào tệp buy_data.json")


In [4]:
#crawl data - renting
import requests
from bs4 import BeautifulSoup
import json
import re

# Hàm để lấy dữ liệu từ URL và trích xuất thông tin từ HTML
def extract_data_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }  # Set a User-Agent header to mimic a real browser

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
    
            # Helper function to extract text or return None
            def get_text_or_none(element):
                return element.text.strip() if element else None

            title_element = soup.select_one('h1.text-xl.font-medium')
            title = get_text_or_none(title_element)
            
            address_element = soup.find('p', class_='line-clamp-2 underline lg:no-underline')
            address = get_text_or_none(address_element)
            
            price_element = soup.find('h5', class_='text-black-v8 text-xl font-semibold')
            price = get_text_or_none(price_element)            
            
            description_element = soup.find('div', class_='text-base text-black-v8 leading-[1.875rem]')
            allowed_tags = ['br', 'p']
            for tag in description_element.find_all(True):  # Tìm tất cả các thẻ con
                if tag.name not in allowed_tags:  # Nếu tên thẻ không nằm trong danh sách được phép
                    tag.unwrap()  # Loại bỏ thẻ nhưng giữ lại nội dung
            description = description_element.decode_contents()
            
            phone_pattern = r'(\d{3}[\s\.]?\d{3}[\s\.]?\d{3})'  # Mã số điện thoại có 10 hoặc 11 chữ số
            matches = re.findall(phone_pattern, description)

            if matches:
                phone_number = matches[-1]  # Lấy số điện thoại cuối cùng trong danh sách
            else:
                phone_number = '0349208325'
                
            basic_info = {
                info.find('p').text.strip().replace(':', ''): get_text_or_none(info.find('p', class_='font-medium line-clamp-2'))
                for info in soup.find_all('div', class_='flex text-base text-black-v8 py-2 space-x-8')
            }
            
            # Chỉnh sửa phần diện tích
            area = basic_info.get('Diện tích', None)
            area = area.split('(')[0].strip() if area else None
            
            # Lấy link ảnh đầu tiên
            image_element = soup.select_one('div.image-fill-wrapper img')
            image_link = image_element['src'] if image_element else None
            
            data = {
                'title': title,
                'address': address,
                'price': price,
                'description': description,
                'type_estate': basic_info.get('Loại nhà đất', None),
                'type': 'Cho Thuê',
                'area': area,
                'bedroom': basic_info.get('Số phòng ngủ', None),
                'toilet': basic_info.get('Số phòng tắm/ Toilet', None),
                'law': basic_info.get('Giấy tờ pháp lý', None),
                'direction': basic_info.get('Hướng nhà', None),
                'width': basic_info.get('Mặt tiền', None),
                'floor': basic_info.get('Số tầng', None),
                'image_links': image_link,
                'phonenumber': phone_number
            }
            
            return data
        else:
            print(f"Lỗi khi truy cập vào URL: {url}, Status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Lỗi mạng hoặc yêu cầu: {e}")
        return None
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ URL: {url}, Lỗi: {e}")
        return None

# Read URLs from meeyland_urls.txt file
with open('rent_urls.txt', 'r') as file:
    urls = file.read().splitlines()

# Loop through each URL, extract data, and save to the list
data_list = []
for url in urls:
    data = extract_data_from_url(url)
    if data:
        data_list.append(data)

# Write data to JSON file
with open('rent_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu vào tệp rent_data.json")


Lỗi khi trích xuất thông tin từ URL: https://meeyland.com/sieu-hiem-cho-thue-kinh-doanh-kho-xuong-kim-chung-di-trach-hoai-duc-100m2-x5t-chi-10tr-th-1720968648050, Lỗi: 'NoneType' object has no attribute 'find_all'
Lỗi khi trích xuất thông tin từ URL: https://meeyland.com/phong-tro-35m2-km-thang-nay-chi-3tr-tai-250-80-phan-trong-tue-thanh-tri-full-do-tang-1tr-1720917476260, Lỗi: 'NoneType' object has no attribute 'find_all'
Dữ liệu đã được lưu vào tệp rent_data.json
